# CIFAR-10 classifier from nnBuilder

This is a simple implementation of a CIFAR-10 classifier using the nnBuilder framework. Trying to reproduce the network of https://www.tensorflow.org/tutorials/deep_cnn/ (Work in progress)

In [1]:
import tensorflow as tf
import numpy as np
import sys
sys.path.append('../nnBuilder')
from nnLayer import *
from nnInput import *
from nnTrainer import *
from nnHandler import *
from _nnUtils import *

In [2]:
data=Layer(x=None,type="CIFAR_10")
preprocess_layers=[]
preprocess_layers.append(dict(type="Batch_Random_Crop",shape=[24,24,3]))
preprocess_layers.append(dict(type="Batch_Random_Horizontal_Flip"))
preprocess_layers.append(dict(type="Batch_Random_Brightness"))
preprocess_layers.append(dict(type="Batch_Random_Contrast"))
preprocess_layers.append(dict(type="Batch_Whitening"))
batch=Layer(x=data,type="Batch_Slice",batch=128,out_features=None)#=preprocess_layers)#Need Input pipeline, too slow
layers=[]
layers.append(dict(type="Convolution",pad="SAME",window=5,stride=1,size=64,relu=True,out_features=[
              dict(type="Pool",pad="SAME",window=3,stride=2,pool_type="max"),
              dict(type="Local_Response_Norm")]))
layers.append(dict(type="Convolution",pad="SAME",window=5,stride=1,size=64,relu=True,out_features=[
              dict(type="Pool",pad="SAME",window=3,stride=2,pool_type="max"),
              dict(type="Local_Response_Norm")]))
layers.append(dict(type="Relu",size=384))
layers.append(dict(type="Relu",size=192))
layers.append(dict(type="Linear",size=10,in_features=["Dropout"]))
network_def=dict(type="Network",layers=layers)
network=Layer(x=batch,**network_def)
trainer=ClassifierTrainer(network=network,optimizer="adam",finish=True)
sess=SessManager(data,batch,network,trainer)#,input_network)
sess.start()

Starting new session


In [3]:
trainer.train(0,keep_rate=1.)
trainer.eval_error()
for i in range(10):
    trainer.train(100,keep_rate=1.)
    trainer.train(0,keep_rate=1.)
    trainer.eval_error() #Training error, reaches about 10%

Error rate: 0.896875
Error rate: 0.622656
Error rate: 0.515625
Error rate: 0.510938
Error rate: 0.438281
Error rate: 0.442969
Error rate: 0.426562
Error rate: 0.360156
Error rate: 0.399219
Error rate: 0.403125
Error rate: 0.367188


In [1]:
network.save() #The full network definition
save=network.save() #The full network definition
assert(Layer(x=batch,**save).save()==save) #Consistency
print(save==network_def) #Saving adds stuff, should still be equivalent
save

NameError: name 'network' is not defined

In [3]:
4//2

2